In [1]:
# append sys path
import sys
import os
sys.path.append(os.path.abspath(os.path.join('../..')))

In [2]:
# import modules
import fastparquet as fp
import pandas as pd

In [45]:
# data loader
tickers = fp.ParquetFile('../../data/btcusdt_230518/tickers_230518.parquet').to_pandas()
trades = fp.ParquetFile('../../data/btcusdt_230518/trades_230518.parquet').to_pandas()

In [46]:
def trades_resample(trades, freq='100ms'):
    """Resample trades data to a given frequency"""
    trades['datetime'] = pd.to_datetime(trades['datetime'])
    trades.set_index('datetime', inplace=True)
    trades['side_mapped'] = trades['side'].map(lambda x: -1 if x else 1)

    def sum_amount(x):
        return (x['amount'] * x['side_mapped']).sum()
    
    agg_df = trades.resample(freq).apply({'price': 'mean', 'amount': sum_amount, 'side': 'any'})
    agg_df['side'] = agg_df['amount'] < 0

    return agg_df


In [47]:
trades100ms = trades_resample(trades)
trades100ms

KeyError: 'amount'

In [37]:
trades.columns

Index(['price', 'amount', 'side', 'side_mapped'], dtype='object')